In [1]:
%matplotlib inline
import matplotlib
import seaborn as sns
matplotlib.rcParams['savefig.dpi'] = 144

In [55]:
from spacy.en import English
nlp = English()

In [4]:
import gzip
import ujson as json
with gzip.open('/Volumes/LaCie/Datasets/Amazon Reviews/reviews_Books_5.json.gz') as f:
    data = [json.loads(line) for line in f]

In [10]:
print(data[68016])

{'unixReviewTime': 1307750400, 'reviewTime': '06 11, 2011', 'summary': 'Glad I read it at last', 'reviewerName': 'GVC', 'reviewText': "I had noticed this book before but cast it aside thinking it was another book about the good work missionaries do in the underdeveloped world. How wrong I was.The Poisonwood Bible contains a strong message about all the negatives that can happen with missionary activity.The book tells the story of Nathan Price and his family and what happened when they went to spread the word in the Congo. Things went wrong from the start when arrogant  Pastor Nathan ( wrong person for the job and his church tried to tell him so ) arrived with a basket of seeds from plants native to North America. The Congolese apparently are so primitive they didn't know how to grow their own food. The plants failed in the Congo and Nathan began a difficult learning experience.The brilliance of the book comes from how it operates on so many levels. The story is told through the eyes of

In [11]:
ratings = [row['overall'] for row in data]

In [15]:
print(ratings[1050:1150])

[5.0, 3.0, 1.0, 5.0, 5.0, 5.0, 5.0, 5.0, 4.0, 4.0, 5.0, 5.0, 5.0, 5.0, 4.0, 5.0, 3.0, 5.0, 4.0, 5.0, 5.0, 1.0, 5.0, 2.0, 4.0, 4.0, 4.0, 2.0, 5.0, 4.0, 4.0, 5.0, 5.0, 2.0, 5.0, 4.0, 5.0, 5.0, 5.0, 4.0, 2.0, 4.0, 3.0, 5.0, 5.0, 5.0, 5.0, 4.0, 5.0, 5.0, 5.0, 5.0, 5.0, 5.0, 5.0, 5.0, 4.0, 5.0, 5.0, 5.0, 4.0, 5.0, 5.0, 5.0, 4.0, 4.0, 5.0, 5.0, 4.0, 5.0, 5.0, 4.0, 5.0, 5.0, 5.0, 3.0, 5.0, 5.0, 5.0, 5.0, 4.0, 5.0, 4.0, 5.0, 5.0, 5.0, 5.0, 4.0, 2.0, 4.0, 4.0, 4.0, 5.0, 4.0, 4.0, 5.0, 5.0, 3.0, 5.0, 5.0]


In [20]:
from sklearn import base

#Column selector
class ColumnSelectTransformer(base.BaseEstimator, base.TransformerMixin):
    def __init__(self, col_names):
        self.col_names = col_names
    def fit(self, X, y=None):
        return self
    def transform(self, X):
        return [row['reviewText'] for row in X]

In [17]:
sample = [data[i] for i in range(len(data)) if i%20==0]
sample_ratings = [ratings[j] for j in range(len(data)) if j%20==0]

In [27]:
from sklearn.model_selection import train_test_split

data_train, data_test, ratings_train, ratings_test = train_test_split(data, ratings, test_size=0.2, random_state=42)
len(data), len(data_train), len(data_test)

(8898041, 7118432, 1779609)

In [35]:
sample_data_train, sample_data_test, sample_ratings_train, sample_ratings_test = train_test_split(sample, sample_ratings, test_size=0.2, random_state=42)
len(sample), len(sample_data_train), len(sample_data_test)

(444903, 355922, 88981)

In [36]:
cst = ColumnSelectTransformer(['reviewText'])
print(cst.fit_transform(sample_data_train[:3]))

["The author explains most things very clearly.  I sometimes wish things were organized a little differently or introduced more explicitly for the sake of clarity.  Also, there are some errors that can make things confusing.  (Like an answer that doesn't match the question asked, or an incorrect formula here and there [The formula for carbonate on p 147 is incorrect.])  Mostly a helpful book, though, especially if used alongside actual class coursework.", "Very enjoyable! As a fifty shades fan I would recommend this. It has a much better storyline to it with more in-depth characters. I'm interested to see where the 3rd book will take us", 'No issues at all with the purchase and or the delivery of the item. It has been enjoyed by the recipient. This is a great coffee table book and conversation starter for those mythology lovers in your life']


In [56]:
def tokenize_lemma(text):
    return [w.lemma_ for w in nlp(text)]

In [57]:
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LinearRegression, RidgeCV, Ridge
from sklearn.feature_extraction.text import HashingVectorizer, TfidfVectorizer
from spacy.en.language_data import STOP_WORDS

bag_of_words_est = Pipeline([
    ('ColumnSelect', ColumnSelectTransformer(['reviewText'])),
    #Vectorizer
    ('TfidfVectorizer', TfidfVectorizer(ngram_range=(1,2), 
                                        stop_words=STOP_WORDS,
                                        tokenizer=tokenize_lemma
                                            )),
    #Regressor
    ('Ridge', Ridge())
])

bag_of_words_est.fit(sample_data_train, sample_ratings_train)

Pipeline(steps=[('ColumnSelect', ColumnSelectTransformer(col_names=['reviewText'])), ('TfidfVectorizer', TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=...it_intercept=True, max_iter=None,
   normalize=False, random_state=None, solver='auto', tol=0.001))])

In [58]:
bag_of_words_est.predict(sample_data_test)
bag_of_words_est.score(sample_data_test, sample_ratings_test)

0.45537222802678212